In [58]:

import pandas as pd
import plotly.express as px
import altair as alt
# from sqlalchemy import create_engine
# import secrets
import numpy as np
from datetime import datetime

import plotly.graph_objects as go

In [59]:
df = pd.read_csv(r"data/all_sales_data.csv")
# ### MASTER DATA ###
all_sales = df.copy()

all_sales = all_sales[(all_sales.market_segment != 'Samples')]

# invoice date cleanup
all_sales['date'] = pd.to_datetime(all_sales['date'])
all_sales['date'] = all_sales['date'].dt.normalize()
all_sales['date'] = all_sales['date'].dt.floor('D')

df_selection = all_sales.copy().drop(columns=['month','year','item','qty','cad'])
df_selection

,date,sale_origin,market_segment,parent_customer,customer,usd
0,2024-02-21,dot,Vending,Vistar,VISTAR-FLORIDA,1128.0000
1,2024-02-21,unl,Convenience,Direct University,Colorado State University,0.0000
2,2024-02-21,unl,Convenience,Direct University,Colorado State University,846.0000
3,2024-02-21,unl,Convenience,Direct University,Colorado State University,846.0000
4,2024-02-21,unl,Convenience,Direct University,Colorado State University,846.0000
...,...,...,...,...,...,...
143926,2021-01-04,unl,Online,Shopify,Shopify Customer - AWAKE Energy USA,3.9825
143927,2021-01-04,unl,Online,Amazon,Amazon.com,99.5850
143928,2021-01-04,unl,Online,Shopify,Shopify Customer - AWAKE Energy USA,25.6425
143929,2021-01-04,unl,Online,Shopify,Shopify Customer - AWAKE Energy USA,0.0000


In [61]:
############ MARKET SEGMENT COLOR DICTIONARY

market_segment_color = {
    'Vending': 'rgb(56,149,73)',
    'Grocery': 'rgb(248,184,230)',
    'Alternate Retail': 'rgb(46,70,166)',
    'Canada': 'rgb(204,208,221)',
    'Online': 'rgb(106,87,63)',
    'Other': 'rgb(200,237,233)',
    'Convenience': 'rgb(233,81,46)',
    'Broadline Distributor': 'rgb(233,152,19)',
    'Samples': 'rgb(141,62,92)'}

sale_origin_dict = {
    'Dot': 'rgb(81, 121, 198)',
    'Unleashed': 'rgb(239, 83, 80)'
}

In [60]:
l1 = pd.read_csv(r"data/l1.csv")

# l1 cleanup for bar
l1['usd'] = l1['sub_total']*.75
l1 = l1[(l1.customer_type != 'Samples')][['completed_date','usd']].sort_values(by='completed_date',ascending=False)

l1['completed_date'] = pd.to_datetime(l1['completed_date'])
l1['year'] = l1['completed_date'].dt.year
l1['YearMonth'] = l1['completed_date'].astype('string').apply(lambda x: x.split("-")[0] + "-" + x.split("-")[1])

l1 = l1.groupby(['year','YearMonth'],as_index=False)['usd'].sum()

l1.loc[len(l1)] = [2024, '2024-03', 0]  # zero sales for future months of year
l1.loc[len(l1)] = [2024, '2024-04', 0]
l1.loc[len(l1)] = [2024, '2024-05', 0]
l1.loc[len(l1)] = [2024, '2024-06', 0]
l1.loc[len(l1)] = [2024, '2024-07', 0]
l1.loc[len(l1)] = [2024, '2024-08', 0]
l1.loc[len(l1)] = [2024, '2024-09', 0]
l1.loc[len(l1)] = [2024, '2024-10', 0]
l1.loc[len(l1)] = [2024, '2024-11', 0]
l1.loc[len(l1)] = [2024, '2024-12', 0]

l1_23 = l1[l1.year==2023].usd
l1_24 = l1[l1.year==2024].usd
goal = l1_23 * 1.5

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

color = l1.year.astype('category').unique()

# L1 monthly
fig = go.Figure(
    data=[
        go.Scatter(x=months, y=goal, name="Direct Goal", mode='markers', marker_size=8, marker_line=dict(width=1,color='darkslategrey'),marker_color='orange',hovertemplate="<br>".join(["%{y:.2s}"])),
        go.Bar(x=months, y=l1_23, name="2023", marker_color="#5a5856", marker_opacity=.5,hovertemplate="<br>".join(["%{y:.2s}"])),
        go.Bar(x=months, y=l1_24, name="2024", marker_color='#E09641',hovertemplate="<br>".join(["%{y:.2s}"]))
    ],
    layout=dict(#title='2024', title_x=.45, 
                height=350, barmode='group', template='plotly_white', 
                hoverlabel=dict(font_size=18,font_family="Rockwell"),
                legend=dict(x=0.02, y=1.1, orientation='h'),
                bargap=0.15,bargroupgap=0.1)
)

fig.update_traces(texttemplate='%{y:.2s}')
fig.update_xaxes(showticklabels=True,showgrid=False,gridcolor="#B1A999", tickfont=dict(color='#5A5856', size=14),title_font=dict(color='#5A5856',size=15))
fig.update_yaxes(showticklabels=False,showgrid=True,gridcolor="#B1A999")

In [63]:
df = df_selection.copy()

## add year/month column to help with charting

def getYearMonth(s):
  return (s.split("-")[0] + "-" + s.split("-")[1])

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name()
df['YearMonth'] = df['date'].astype('string').apply(lambda x: getYearMonth(x))

# group by month year and add $0 sales to future months
chart_df = round(df.groupby(['year','YearMonth'],as_index=False)['usd'].sum())

chart_df.loc[len(chart_df)] = [2024, '2024-03', 0]
chart_df.loc[len(chart_df)] = [2024, '2024-04', 0]
chart_df.loc[len(chart_df)] = [2024, '2024-05', 0]
chart_df.loc[len(chart_df)] = [2024, '2024-06', 0]
chart_df.loc[len(chart_df)] = [2024, '2024-07', 0]
chart_df.loc[len(chart_df)] = [2024, '2024-08', 0]
chart_df.loc[len(chart_df)] = [2024, '2024-09', 0]
chart_df.loc[len(chart_df)] = [2024, '2024-10', 0]
chart_df.loc[len(chart_df)] = [2024, '2024-11', 0]
chart_df.loc[len(chart_df)] = [2024, '2024-12', 0]

chart = chart_df.copy()

# variables for calcs, hovers, etc

sales_22 = chart_df[chart_df.year==2022].usd
sales_23 = chart_df[chart_df.year==2023].usd
sales_24 = chart_df[chart_df.year==2024].usd

goal = sales_23 * 1.5

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

color = chart_df.year.astype('category').unique()


# Bar and scatter monthly

fig = go.Figure(
    data=[
        go.Scatter(x=months, y=goal, name="Goal", mode='markers', marker_size=8, marker_line=dict(width=1,color='darkslategrey'),marker_color='orange',hovertemplate="<br>".join(["%{y:.2s}"])),
        go.Bar(x=months, y=sales_23, name="2023", marker_color="#5a5856", marker_opacity=.5,hovertemplate="<br>".join(["%{y:.2s}"])),
        go.Bar(x=months, y=sales_24, name="2024", marker_color='#E09641',hovertemplate="<br>".join(["%{y:.2s}"]))
    ],
    layout=dict(title='TRUE Sales YoY', title_x=.45, 
                height=350, barmode='group', template='plotly_white', 
                hoverlabel=dict(font_size=18,font_family="Rockwell"),
                legend=dict(x=0, y=1.1, orientation='h'),
                bargap=0.15,bargroupgap=0.1)
)

fig.update_traces(texttemplate='%{y:.2s}')
fig.update_xaxes(showticklabels=True,showgrid=False,gridcolor="#B1A999", tickfont=dict(color='#5A5856', size=14),title_font=dict(color='#5A5856',size=15))
fig.update_yaxes(showticklabels=False,showgrid=True,gridcolor="#B1A999")

fig.show()

In [70]:
l1_23.sum()

9960212.1225

In [65]:
sales_23.sum()

9612344.0

In [ ]:
# DAILY SALES HORIZONTAL SCATTER (XAXIS = CATEGORY)

df = df_selection.groupby([df_selection.date,'market_segment','parent_customer']).usd.sum().reset_index()

#all sales weekly scatter
fig_scatter_all = px.scatter(
    round(df[df.date>'2024-01-31'].sort_index()),
    x='date',
    y='usd',
    template = 'simple_white',
    labels={'date':'',
            'usd':'<b>$USD</b>'},
    height=500,
    color='market_segment',
    color_discrete_map=market_segment_color,
    title='Daily Sales by Market Segment',
    log_y=True
)

# fig_scatter_all.update_traces(name="",hovertemplate="<b>%{x}<br>%{y:$,.0f}")
fig_scatter_all.update_traces(marker=dict(size=18,opacity=.7,line=dict(width=1,color='grey')),selector=dict(mode='markers'))

fig_scatter_all.update_coloraxes(showscale=False)
fig_scatter_all.update_yaxes(gridcolor='lightgray', tickvals=[100,1000,10000,100000])

fig_scatter_all.update_layout(hoverlabel=dict(font_size=15,font_family="Rockwell"))

In [ ]:
# WEEKLY SALES HORIZONTAL SCATTER (XAXIS = CATEGORY)

df_selection.index = pd.to_datetime(df_selection['date'],format = '%y/%m')
sales_per_week = df_selection.groupby([pd.Grouper(freq='W'),'market_segment'])['usd'].sum().reset_index()
df = sales_per_week.copy()

#all sales weekly scatter
fig_scatter_all = px.scatter(
    round(df[df.date>'2023-12-31'].sort_index()),
    x='date',
    y='usd',
    template = 'simple_white',
    labels={'date':'',
            'usd':'<b>$USD</b>'},
    height=500, color='market_segment',
    color_discrete_map=market_segment_color,
    title='Weekly Sales by Market Segment',
    log_y=True
)

fig_scatter_all.update_traces(name="",hovertemplate="<b>%{x}<br>%{y:$,.0f}")
fig_scatter_all.update_traces(marker=dict(size=15,line=dict(width=1,color='grey')),selector=dict(mode='markers'))

fig_scatter_all.update_coloraxes(showscale=False)
fig_scatter_all.update_yaxes(gridcolor='lightgray', tickvals=[100,1000,10000,100000,1000000])

fig_scatter_all.update_layout(hoverlabel=dict(font_size=15,font_family="Rockwell"))

In [ ]:
# MONTHLY SALES HORIZONTAL SCATTER (XAXIS = CATEGORY)

df_selection.index = pd.to_datetime(df_selection['date'],format = '%y/%m')
sales_per_month = df_selection.groupby([pd.Grouper(freq='M'),'market_segment'])['usd'].sum().reset_index()
# df = sales_per_month.copy()

#all sales weekly scatter
fig_scatter_all = px.scatter(
    round(sales_per_month),
    x='date',
    y='usd',
    template = 'simple_white',
    labels={'date':'',
            'usd':'<b>$USD</b>'},
    height=500,
#     size='usd',
#     size_max=15,
    color='market_segment',
    color_discrete_map=market_segment_color,
    title='Monthly Sales by Market Segment',
    trendline="lowess", trendline_options=dict(frac=0.5)
).update_traces(name="",hovertemplate="<b>%{y:$,.0f}")

fig_scatter_all.update_coloraxes(showscale=False)
fig_scatter_all.update_yaxes(gridcolor='lightgray')

fig_scatter_all.update_layout(hoverlabel=dict(font_size=15,font_family="Rockwell"))

In [ ]:
# WEEKLY SALES HORIZONTAL SCATTER (XAXIS = DATETIME)

df_selection.index = pd.to_datetime(df_selection['date'],format = '%y/%m/%d')
sales_per_week = df_selection.groupby(pd.Grouper(freq='W'))['usd'].sum()
sales_per_week = df_selection.copy()

#all sales weekly scatter
fig_scatter_all = px.scatter(
    round(sales_per_week),
    x=sales_per_week.index,
    y='usd',
    template = 'presentation',
    labels={'date':'',
            'usd':'<b>$USD</b>'},
    height=500,
#     size='usd',
    size_max=15,
    color='usd',
    color_continuous_scale=px.colors.sequential.Oranges,
    trendline="rolling", trendline_options=dict(function="mean", window=10), trendline_scope="overall", trendline_color_override="grey"
).update_traces(name="",hovertemplate="Sales: <b>%{y:$,.0f}")
fig_scatter_all.update_coloraxes(showscale=False)
fig_scatter_all.update_yaxes(gridcolor='lightgray')
fig_scatter_all.update_layout(
    hovermode='x unified',
    legend=dict(y=0.99, x=0.1,title='10-Wk Moving Avg'))
fig_scatter_all.update_layout(hoverlabel=dict(font_size=15,font_family="Rockwell"))#,xaxis=dict(type = 'category'))
# fig_scatter_all.update_xaxes(tickmode='array',tickvals = sales_per_week.index, ticktext=sales_per_week.index.month_name())

In [ ]:
# segment pie chart

seg_sales = round(df_selection.groupby('market_segment',as_index=False)['usd'].sum()).sort_values(by='market_segment',ascending=False)

fig_seg_sales = px.pie(
    seg_sales, 
    values='usd', 
    names='market_segment',
    template = 'simple_white',
    title=' ',
    opacity = .8,
    hole=.33,
    color='market_segment',
    color_discrete_map=market_segment_color).update_layout(autosize=False,width=450,height=450,showlegend=False)
fig_seg_sales.update_traces(textposition='inside', textinfo='percent+label', texttemplate='%{label}<br>%{percent:.0%}')
# fig_seg_sales.update_traces(hovertemplate='%{names}<br>%{values:.0%}')
fig_seg_sales.show()

In [ ]:
## FUNCTION TAKES IN DF_SELECTION AND OUTPUTS DAILY SALES DATAFRAME

def make_daily_bar_df(df_selection):
    return (
        df_selection.dropna()
        .assign(new_date=pd.to_datetime(df_selection['date'])).reset_index(drop=True)
        .drop(columns=['cad','month','year','date'])
        .rename(columns={'new_date':'date'})
        .groupby(['market_segment','date','sale_origin','parent_customer'],as_index=False)
        .sum()
        .set_index('date')
        .sort_index()
    )

daily_bar_df = make_daily_bar_df(df_selection)
daily_bar_df

In [ ]:
## DAILY SALES DATAFRAME (SAME AS FUNC ABOVE)

df_selection.groupby([df_selection.date,'sale_origin','market_segment','parent_customer']).usd.sum().reset_index().set_index('date')

In [ ]:
###  DAILY SALES VERTICAL SCATTER (3 WAYS)

# for 'sales_origin':
#   color_by: sale_origin
#   color_discrete_map =  {'unl':"#E62F29",'dot':"#3A4DA1"}
#   marker size: 20

# for 'market_segment':
#   color_by: market_segment
#   color discrete_map = market_segment_color dictionary
#   marker size: 15

# for 'parent_customer':
#   color_by: market_segment
#   color discrete_map = market_segment_color dictionary
#   marker size: 12



week_ago = datetime.today().date() - pd.offsets.Day(10)
df_selection = df_selection[df_selection.date>week_ago]
df = df_selection.groupby([df_selection.date,'sale_origin']).usd.sum().reset_index().set_index('date')
fig = px.scatter(df,
           x='usd',
           color='sale_origin',
           color_discrete_map={'unl':"#E62F29",'dot':"#3A4DA1"},
           opacity=.8,
           labels={'date':"", 'usd':''},
           height=750,
           width=400,
           template='plotly_white',
           title='Sales Last 10 Days',
           hover_name='sale_origin',
          )
fig.update_traces(marker=dict(size=20,line=dict(width=1,color='grey')),selector=dict(mode='markers'))
fig.update_xaxes(showgrid=True,tickprefix='$',gridcolor="#B1A999",tickfont=dict(color='#5A5856', size=12),showticklabels=True)
fig.update_yaxes(showgrid=False,gridcolor='gray',tickfont=dict(color='#5A5856', size=14))
fig.update_layout(xaxis={'side':'top'}, title_x=0.33,legend=dict(title=''))
fig.update_layout(yaxis=dict(type = 'category'))
fig.update_yaxes(tickmode='array',tickvals = df.index, ticktext=df.index.strftime('%m-%d'))
fig.show()

In [ ]:
week_ago = datetime.today().date() - pd.offsets.Day(10)
df_selection = df_selection[df_selection.date>week_ago]

fig = px.scatter(df_selection.groupby([df_selection.date,'market_segment']).usd.sum().fillna(0).reset_index().set_index('date'),
           x='usd',
           color='market_segment',
           color_discrete_map=market_segment_color,                   # if color_by: market_segment
           opacity=.8,
           labels={'date':"", 'usd':''},
           height=800,
           width=400,
           template='plotly_white',
           title='Sales Last 10 Days',
           hover_name='market_segment',
           log_x=True,
          ).update_layout(showlegend=False)
fig.update_traces(marker=dict(size=15,line=dict(width=1,color='grey')),selector=dict(mode='markers'))
fig.update_xaxes(showgrid=True,tickprefix='$',gridcolor="#B1A999",tickfont=dict(color='#5A5856', size=12), tickvals=[100,1000,10000,100000],showticklabels=True)
fig.update_yaxes(showgrid=False,gridcolor='gray',tickfont=dict(color='#5A5856', size=14))
fig.update_layout(xaxis={'side':'top'},title_x=0.33)#,legend=dict(title=''))    # if you add legend turn this on
fig.update_yaxes(tickmode='array',tickvals = df.index, ticktext=df.index.strftime('%m-%d'))


fig.show()

In [ ]:
week_ago = datetime.today().date() - pd.offsets.Day(10)
df_selection = df_selection[df_selection.date>week_ago]

fig = px.scatter(df_selection.groupby([df_selection.date,'parent_customer','market_segment']).usd.sum().reset_index().set_index('date'),
           x='usd',
           color='market_segment',
           color_discrete_map=market_segment_color,
           opacity=.8,
           labels={'date':"", 'usd':''},
           height=800,
           width=400,
           template='plotly_white',
           title='Sales Last 10 Days',
           hover_name='parent_customer',
           log_x = True
          ).update_layout(showlegend=False)
fig.update_traces(marker=dict(size=12,line=dict(width=1,color='grey')),selector=dict(mode='markers'))
fig.update_xaxes(showgrid=True,tickprefix='$',gridcolor="#B1A999",tickfont=dict(color='#5A5856', size=12),showticklabels=True)
fig.update_yaxes(showgrid=False,gridcolor='gray',tickfont=dict(color='#5A5856', size=14))
fig.update_layout(xaxis={'side':'top'}, title_x=0.33,legend=dict(title=''))
fig.update_yaxes(tickmode='array',tickvals = df.index, ticktext=df.index.strftime('%m-%d'))

fig.show()

In [ ]:
df = all_sales.groupby([all_sales.date,'market_segment']).usd.sum().reset_index().set_index('date')
df = df[df.index>'2023-12-31'].pivot(columns='market_segment', values='usd')

fig = px.area(df,
              color='market_segment',
              color_discrete_map=market_segment_color, 
              facet_col="market_segment",facet_col_wrap=2, 
              height=1000,
              template = 'plotly_white',
              title='YTD Sales by Segment'
             )
fig.update_traces(fill='tonexty')
fig.show()

In [ ]:
df = df_selection[df_selection.market_segment=='Vending']['usd'].reset_index().set_index('date')
df = df[df.index>'2023-12-31']

fig = px.area(df,
              template = 'plotly_white',
              title='YTD Sales by Segment'
             )
fig.show()

In [ ]:
df_selection.groupby([df_selection.date,'market_segment']).usd.sum().reset_index().set_index('date')

In [ ]:
# DAILY SALES HORIZONTAL SCATTER
df = all_sales.groupby([all_sales.date,'market_segment']).usd.sum().reset_index().set_index('date')
df = round(df[df.index>'2024-01-31'].sort_index())

fig_scatter_all = px.scatter(
    df,
    y='usd',
    template = 'plotly_white',
    labels={'date':'FEBRUARY',
            'usd':''},
    height=325,
    color='market_segment',
    color_discrete_map=market_segment_color,
    # title='February',
    log_y=True
)
fig_scatter_all.show()